In [1]:
import numpy as np   # 수 연산
import pandas as pd   # 데이터분석
import matplotlib.pyplot as plt    # 시각화
import seaborn as sns   # 샘플 + 화려함

import matplotlib.font_manager as fm # 한글폰트를 쓰기 위한 방법(?)
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import operator
from numpy import linalg

In [3]:
#머신러닝 관련(사이킷런)
import mglearn

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve
from sklearn.metrics import precision_score, recall_score

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import Perceptron

from sklearn.svm import LinearSVR, SVR, SVC

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance, plot_tree

from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import plot_importance, plot_tree, plot_metric

import statsmodels.api as sm

# 구내식당 중식 인원 머신러닝(Datetime data deleted)

+ 1. Use train data and check performance of each algorithm
+ 2. Apply some best algorithm to test data

In [117]:
ln_train = pd.read_csv("../byun/data/lunch_df_encoding.csv", encoding="utf-8")
ln_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


In [118]:
ln_train.info()
ln_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               1205 non-null   object 
 1   year                   1205 non-null   int64  
 2   month                  1205 non-null   int64  
 3   date                   1205 non-null   int64  
 4   worker_number          1205 non-null   int64  
 5   real_number            1205 non-null   int64  
 6   vacation_number        1205 non-null   int64  
 7   biztrip_number         1205 non-null   int64  
 8   overtime_number        1205 non-null   int64  
 9   telecom_number         1205 non-null   int64  
 10  temperature            1204 non-null   float64
 11  rain                   1205 non-null   float64
 12  wind                   1204 non-null   float64
 13  humidity               1205 non-null   float64
 14  discomfort_index       1204 non-null   float64
 15  perc

(1205, 31)

In [119]:
ln_train.isnull().sum() #칼럼별 결측치 수

datetime                 0
year                     0
month                    0
date                     0
worker_number            0
real_number              0
vacation_number          0
biztrip_number           0
overtime_number          0
telecom_number           0
temperature              1
rain                     0
wind                     1
humidity                 0
discomfort_index         1
perceived_temperature    2
lunch_rice               0
lunch_number             0
season_fall              0
season_spring            0
season_summer            0
season_winter            0
weekdays_friday          0
weekdays_monday          0
weekdays_thursday        0
weekdays_tuesday         0
weekdays_wednesday       0
vacation_N               0
vacation_Y               0
new_lunch_N              0
new_lunch_Y              0
dtype: int64

In [120]:
ln_train = ln_train.dropna()
ln_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


## 선형회귀(일반) - train: 78.40%, test: 81.09%

In [121]:
ln_X= ln_train.drop(columns=["datetime", "lunch_number"], axis=1)

ln_y = ln_train["lunch_number"]

ln_y = ln_y.to_numpy()

X.shape

(1203, 29)

In [122]:
lr = LinearRegression().fit(ln_X, ln_y)
print("훈련 점수 : ", lr.score(ln_X, ln_y))

훈련 점수 :  0.7743430539169095


In [123]:
ln_X_train, ln_X_test, ln_y_train, ln_y_test = train_test_split(ln_X, ln_y, test_size=0.2, random_state=0)
lr = LinearRegression().fit(ln_X_train, ln_y_train)

In [124]:
print("훈련 점수 : ", lr.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", lr.score(ln_X_test, ln_y_test))

훈련 점수 :  0.7840493791824227
테스트 점수 :  0.8109110547383374


## 릿지회귀모델 - train: 78.42%, test: 80.96%

In [125]:
ridge = Ridge().fit(ln_X_train, ln_y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", ridge.score(ln_X_test, ln_y_test))

ridge10 = Ridge(alpha=10).fit(ln_X_train, ln_y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge10.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", ridge10.score(ln_X_test, ln_y_test))


ridge01 = Ridge(alpha=0.000000001).fit(ln_X_train, ln_y_train) # Ridge(alpha=1.0) 이 디폴트

print("훈련 점수 : ", ridge01.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", ridge01.score(ln_X_test, ln_y_test))

# Ridge: lower alpha makes greater score

훈련 점수 :  0.784169984263926
테스트 점수 :  0.8095359944176708
훈련 점수 :  0.7832749706416915
테스트 점수 :  0.807618285322794
훈련 점수 :  0.7841815322653001
테스트 점수 :  0.8096388073522368


## 라쏘회귀모델 - train: 78.41%, test: 80.95%

In [126]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(ln_X_train, ln_y_train) #Lasso(alpha=1) 디폴트

print("훈련 점수 : ", lasso.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", lasso.score(ln_X_test, ln_y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso.coef_ != 0)) 
lasso001 = Lasso(alpha=0.01).fit(ln_X_train, ln_y_train)

print("훈련 점수 : ", lasso001.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", lasso001.score(ln_X_test, ln_y_test))

print("사용한 특성의 수 : ", np.sum(lasso001.coef_ != 0))

lasso00001 = Lasso(alpha=0.000000001).fit(ln_X_train, ln_y_train)

print("훈련 점수 : ", lasso00001.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", lasso00001.score(ln_X_test, ln_y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso00001.coef_ != 0))

# Lasso: lower alpha makes greater score

훈련 점수 :  0.7800909771016679
테스트 점수 :  0.8024396252730003
사용한 특성의 수 :  23
훈련 점수 :  0.784165574968005
테스트 점수 :  0.8094858319521451
사용한 특성의 수 :  25
훈련 점수 :  0.7841678420259279
테스트 점수 :  0.8095114403342413
사용한 특성의 수 :  27


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 58403.39447391033, tolerance: 4229.542462474013
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4195097.303080663, tolerance: 4229.542462474013
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4564356.348157798, tolerance: 4229.542462474013
  model = cd_fast.enet_coordinate_descent(


## 엘라스틱넷 - train: 77.43% , test: 80.50%

In [127]:
param = {'alpha':[0.01, 0.1, 1, 10, 100]}

model = ElasticNet()
gs= GridSearchCV(estimator=model, param_grid=param, scoring='neg_mean_squared_error',cv=5)

gs.fit(ln_X_train, ln_y_train)
print(gs.best_estimator_)

C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3542937.5427042334, tolerance: 3355.7003799739928
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3635067.6762525844, tolerance: 3337.5886241872568
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3663052.039123235, tolerance: 3448.8833116883116
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Conv

ElasticNet(alpha=0.01)


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4429321.143763017, tolerance: 4229.542462474013
  model = cd_fast.enet_coordinate_descent(


In [128]:
model = ElasticNet(alpha=0.01)
model.fit(ln_X_train, ln_y_train)

print("훈련 점수 : ", model.score(ln_X_train, ln_y_train))
print("테스트 점수 : ", model.score(ln_X_test, ln_y_test))

훈련 점수 :  0.7839200028188942
테스트 점수 :  0.808786996578159


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4429321.143763017, tolerance: 4229.542462474013
  model = cd_fast.enet_coordinate_descent(


## SVR model - train: 77.69%, test: 80.43% (단, 표준화 했을 때)

- Normal
- Kernel("linear" is best)
- StandardScaler // MinMaxScaler

In [129]:
model = SVR()  # default is rbf
model.fit(ln_X_train, ln_y_train)

print("학습 결과 : ", model.score(ln_X_train, ln_y_train))
print("테스트 결과 : ", model.score(ln_X_test, ln_y_test))

# Too bad

학습 결과 :  0.014302357832055801
테스트 결과 :  0.018706671477887626


In [130]:
linear_svr = SVR(kernel="linear")
linear_svr.fit(ln_X_train, ln_y_train)

print("학습 결과 : ", linear_svr.score(ln_X_train, ln_y_train))
print("테스트 결과 : ", linear_svr.score(ln_X_test, ln_y_test))

print('===================================================')

linear_svr = SVR(kernel="linear", C=10, gamma='scale')
linear_svr.fit(ln_X_train, ln_y_train)

print("학습 결과 : ", linear_svr.score(ln_X_train, ln_y_train))
print("테스트 결과 : ", linear_svr.score(ln_X_test, ln_y_test))

학습 결과 :  0.7550149762612375
테스트 결과 :  0.7683826982336686
학습 결과 :  0.7366317899534491
테스트 결과 :  0.7850674848726336


In [131]:
### 표준화 해보자

scaler = StandardScaler()
ln_X_train_std = scaler.fit_transform(ln_X_train)
ln_X_test_std = scaler.transform(ln_X_test)

model = SVR(kernel='linear')
model.fit(ln_X_train_std, ln_y_train)

print("학습 결과 : ", model.score(ln_X_train_std, ln_y_train))
print("테스트 결과 : ", model.score(ln_X_test_std, ln_y_test))

## 표준화만 했는데도 성능이 좋아짐

학습 결과 :  0.7713697303368211
테스트 결과 :  0.7988608914036323


In [132]:
model = SVR(kernel='linear', C=10)
model.fit(ln_X_train_std, ln_y_train)

print("학습 결과 : ", model.score(ln_X_train_std, ln_y_train))
print("테스트 결과 : ", model.score(ln_X_test_std, ln_y_test))
print('===================================================')
model = SVR(kernel='linear', C=10, gamma='auto')
model.fit(ln_X_train_std, ln_y_train)

print("학습 결과 : ", model.score(ln_X_train_std, ln_y_train))
print("테스트 결과 : ", model.score(ln_X_test_std, ln_y_test))
print('===================================================')
# gamma=scale & gamma=auto are same score

학습 결과 :  0.774621434983584
테스트 결과 :  0.8038651022229766
학습 결과 :  0.774621434983584
테스트 결과 :  0.8038651022229766


In [133]:
# best 값 적용
model = SVR(kernel="linear", C=10.0, gamma="scale", epsilon=10.0)
model.fit(ln_X_train_std, ln_y_train)

print("학습 결과 : ", model.score(ln_X_train_std, ln_y_train))
print("테스트 결과 : ", model.score(ln_X_test_std, ln_y_test))

학습 결과 :  0.7753010816261348
테스트 결과 :  0.802672447156369


## Gradient Boosting Model

- XGBoostRegressor(overfitting)
- LightGBMRegressor(overfitting)

### XGBoostRegressor - train: 88.16%, test: 84.67%, mean_test score: 76.30%

In [134]:
xgbr = XGBRegressor(n_estimators=300, colsample_bylevel=0.5, learning_rate=0.08, 
                    colsample_bytree=0.5, max_depth=2, objective="reg:squarederror")
xgbr.fit(ln_X_train, ln_y_train)

print("훈련 정확도 : ", xgbr.score(ln_X_train, ln_y_train))
print("테스트 정확도 : ", xgbr.score(ln_X_test, ln_y_test))

cross_val = cross_validate(estimator=xgbr, X=ln_X, y=ln_y, cv=10)
print("테스트 정확도 평균 : ", cross_val["test_score"].mean())

훈련 정확도 :  0.8816151969337858
테스트 정확도 :  0.8466628387297592
테스트 정확도 평균 :  0.7629508901837475


### LightGBM Regressor - train: 95.70%, test: 79.84%, testscore_mean: 75.76%

In [135]:
ln_X_train, ln_X_test, ln_y_train, ln_y_test = train_test_split(ln_X, ln_y, test_size=.2, random_state=42)

lgbr = LGBMRegressor(n_estimator=400)
lgbr.fit(ln_X_train, ln_y_train, early_stopping_rounds=150, 
         eval_metric="logloss", eval_set=[(ln_X_test, ln_y_test)], verbose=True)

print("훈련 정확도 : ", lgbr.score(ln_X_train, ln_y_train))
print("테스트 정확도 : ", lgbr.score(ln_X_test, ln_y_test))

cross_val = cross_validate(estimator=lgbr, X=ln_X, y=ln_y, cv=10)
print("테스트 정확도 평균 : ", cross_val["test_score"].mean())

[LightGBM] [Warning] Unknown parameter: n_estimator
[1]	valid_0's l2: 34335.5
[2]	valid_0's l2: 30007.8
[3]	valid_0's l2: 26525.3
[4]	valid_0's l2: 23660.9
[5]	valid_0's l2: 21288.3
[6]	valid_0's l2: 19447.9
[7]	valid_0's l2: 17816.5
[8]	valid_0's l2: 16641.3
[9]	valid_0's l2: 15617.8
[10]	valid_0's l2: 14627.1
[11]	valid_0's l2: 13911.6
[12]	valid_0's l2: 13327.5
[13]	valid_0's l2: 12805.4
[14]	valid_0's l2: 12316.6
[15]	valid_0's l2: 11977.3
[16]	valid_0's l2: 11437.8
[17]	valid_0's l2: 10984
[18]	valid_0's l2: 10590.2
[19]	valid_0's l2: 10287.6
[20]	valid_0's l2: 10050.3
[21]	valid_0's l2: 9850.36
[22]	valid_0's l2: 9663.9
[23]	valid_0's l2: 9474.57
[24]	valid_0's l2: 9324.92
[25]	valid_0's l2: 9318.51


C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[26]	valid_0's l2: 9228.01
[27]	valid_0's l2: 9162.23
[28]	valid_0's l2: 9120.52
[29]	valid_0's l2: 9012.56
[30]	valid_0's l2: 8902.43
[31]	valid_0's l2: 8816.41
[32]	valid_0's l2: 8708.31
[33]	valid_0's l2: 8643.22
[34]	valid_0's l2: 8601.15
[35]	valid_0's l2: 8550.15
[36]	valid_0's l2: 8507.44
[37]	valid_0's l2: 8456.75
[38]	valid_0's l2: 8403.27
[39]	valid_0's l2: 8306.37
[40]	valid_0's l2: 8264.95
[41]	valid_0's l2: 8243.04
[42]	valid_0's l2: 8229.84
[43]	valid_0's l2: 8199.98
[44]	valid_0's l2: 8185.66
[45]	valid_0's l2: 8163.25
[46]	valid_0's l2: 8175.97
[47]	valid_0's l2: 8148.35
[48]	valid_0's l2: 8164.64
[49]	valid_0's l2: 8143.3
[50]	valid_0's l2: 8118.34
[51]	valid_0's l2: 8094.41
[52]	valid_0's l2: 8092.63
[53]	valid_0's l2: 8071
[54]	valid_0's l2: 8056.3
[55]	valid_0's l2: 8019.22
[56]	valid_0's l2: 8035.61
[57]	valid_0's l2: 8024.1
[58]	valid_0's l2: 8020.32
[59]	valid_0's l2: 7976.51
[60]	valid_0's l2: 7978.94
[61]	valid_0's l2: 7971.25
[62]	valid_0's l2: 7956.12
[63]	va

# Lunch Test data

In [136]:
ln_test = pd.read_csv("../byun/data/lunch_test_encoding.csv", encoding="utf-8")
ln_test

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_lunch_N,new_lunch_Y
0,2021-01-27,2021,1,27,2983,2355.0,88,182,5,358.0,...,1,0,0,0,0,1,1,0,1,0
1,2021-01-28,2021,1,28,2983,2319.0,104,212,409,348.0,...,1,0,0,1,0,0,1,0,1,0
2,2021-01-29,2021,1,29,2983,2170.0,270,249,0,294.0,...,1,1,0,0,0,0,1,0,1,0
3,2021-02-01,2021,2,1,2924,2340.0,108,154,538,322.0,...,1,0,1,0,0,0,1,0,1,0
4,2021-02-02,2021,2,2,2924,2362.0,62,186,455,314.0,...,1,0,0,0,1,0,1,0,1,0
5,2021-02-03,2021,2,3,2924,2380.0,59,199,5,286.0,...,1,0,0,0,0,1,1,0,1,0
6,2021-02-04,2021,2,4,2924,2364.0,61,211,476,288.0,...,1,0,0,1,0,0,1,0,1,0
7,2021-02-05,2021,2,5,2924,2247.0,169,252,0,256.0,...,1,1,0,0,0,0,1,0,1,0
8,2021-02-08,2021,2,8,2924,2333.0,88,174,690,329.0,...,1,0,1,0,0,0,1,0,1,0
9,2021-02-09,2021,2,9,2924,2318.0,94,183,542,329.0,...,1,0,0,0,1,0,1,0,1,0


In [137]:
summer = []
fall = []

for i in range(len(ln_test)):
    summer.append(0)
    fall.append(0)
    
ln_test['season_summer'] = summer
ln_test['season_fall'] = fall

ln_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               50 non-null     object 
 1   year                   50 non-null     int64  
 2   month                  50 non-null     int64  
 3   date                   50 non-null     int64  
 4   worker_number          50 non-null     int64  
 5   real_number            50 non-null     float64
 6   vacation_number        50 non-null     int64  
 7   biztrip_number         50 non-null     int64  
 8   overtime_number        50 non-null     int64  
 9   telecom_number         50 non-null     float64
 10  temperature            50 non-null     float64
 11  rain                   50 non-null     float64
 12  wind                   50 non-null     float64
 13  humidity               50 non-null     float64
 14  discomfort_index       50 non-null     float64
 15  perceive

In [138]:
columns = ['datetime', 'year', 'month', 'date', 'worker_number', 'real_number',
       'vacation_number', 'biztrip_number', 'overtime_number',
       'telecom_number', 'temperature', 'rain', 'wind', 'humidity',
       'discomfort_index', 'perceived_temperature', 'lunch_rice',
       'season_fall','season_spring', 'season_summer', 'season_winter', 'weekdays_friday', 'weekdays_monday',
       'weekdays_thursday', 'weekdays_tuesday', 'weekdays_wednesday',
       'vacation_N', 'vacation_Y', 'new_lunch_N', 'new_lunch_Y']

ln_test.coulmns = columns

<ipython-input-138-695c2266ae23>:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ln_test.coulmns = columns


In [139]:
ln_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               50 non-null     object 
 1   year                   50 non-null     int64  
 2   month                  50 non-null     int64  
 3   date                   50 non-null     int64  
 4   worker_number          50 non-null     int64  
 5   real_number            50 non-null     float64
 6   vacation_number        50 non-null     int64  
 7   biztrip_number         50 non-null     int64  
 8   overtime_number        50 non-null     int64  
 9   telecom_number         50 non-null     float64
 10  temperature            50 non-null     float64
 11  rain                   50 non-null     float64
 12  wind                   50 non-null     float64
 13  humidity               50 non-null     float64
 14  discomfort_index       50 non-null     float64
 15  perceive

In [140]:
ln_X_test= ln_test[['year', 'month', 'date', 'worker_number', 'real_number',
       'vacation_number', 'biztrip_number', 'overtime_number',
       'telecom_number', 'temperature', 'rain', 'wind', 'humidity',
       'discomfort_index', 'perceived_temperature', 'lunch_rice',
       'season_fall','season_spring', 'season_summer', 'season_winter', 'weekdays_friday', 'weekdays_monday',
       'weekdays_thursday', 'weekdays_tuesday', 'weekdays_wednesday',
       'vacation_N', 'vacation_Y', 'new_lunch_N', 'new_lunch_Y']]
ln_X_test.shape
ln_X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   50 non-null     int64  
 1   month                  50 non-null     int64  
 2   date                   50 non-null     int64  
 3   worker_number          50 non-null     int64  
 4   real_number            50 non-null     float64
 5   vacation_number        50 non-null     int64  
 6   biztrip_number         50 non-null     int64  
 7   overtime_number        50 non-null     int64  
 8   telecom_number         50 non-null     float64
 9   temperature            50 non-null     float64
 10  rain                   50 non-null     float64
 11  wind                   50 non-null     float64
 12  humidity               50 non-null     float64
 13  discomfort_index       50 non-null     float64
 14  perceived_temperature  50 non-null     float64
 15  lunch_ri

In [141]:
# LR
model = LinearRegression()
model.fit(ln_X, ln_y)

lr_preds = model.predict(ln_X_test)
print(lr_preds)

[ 971.21029282  888.13948822  637.7768774  1264.32562733  996.00293541
  958.9662838   918.45301819  671.881073   1251.66877937  990.26537132
  782.07868004 1301.56736946 1098.94449234  989.26228523  880.31886196
  652.25513268 1234.88637161 1029.35627174  868.1578598   866.85135078
  535.21593952 1066.96072006  954.29671574  940.13217163  671.33915997
 1367.1117878  1137.54461479  958.55778503  910.71105385  691.05289459
 1305.27346611 1029.45988464  928.44533348  860.55181694  673.3137722
 1220.04856491  985.70016384  869.50530815  782.3562603   562.62273788
 1189.99376678  966.03079987  886.20377254  793.99547005  570.87977409
 1213.7815361  1026.90176678  890.72439003  897.63660145  591.34755516]


In [142]:
# SVR
model = SVR(kernel='linear', C=10, gamma='scale', epsilon=10)
model.fit(ln_X, ln_y)

svr_preds = model.predict(ln_X_test)
print(svr_preds)

[ 874.25740351  818.27338915  586.38687025 1192.23411541 1005.25792424
  899.85817205  895.12867116  631.22922121 1239.36062114  979.33480032
  725.48237507 1262.58777024 1092.21614234  931.72606547  852.87381159
  589.62924229 1190.09798338 1042.27126797  797.21308209  796.61087568
  478.94725202 1131.626038    940.35249756  952.11201497  675.34251385
 1374.75897448 1211.04275815  950.10113908  924.92742471  653.73037584
 1325.42829011 1062.03867893  900.65130152  855.73668708  638.00882699
 1232.02560021 1015.64505105  828.24833848  765.55551921  544.5392798
 1183.40371908  993.22290735  845.70428778  804.9845006   585.37785073
 1239.66413243 1078.47246768  879.95480059  889.13305666  597.86730479]


In [143]:
# Ridge
model = Ridge(alpha=0.01)
model.fit(ln_X, ln_y)

rdg_preds = model.predict(ln_X_test)
print(rdg_preds)

[ 958.78772465  878.62893135  627.31201702 1256.87151155  984.93088715
  946.99742164  908.69503526  663.24529808 1238.75199701  979.03359005
  772.12793992 1298.22751645 1093.51681144  976.34637252  874.15118043
  642.15440344 1222.75245351 1015.07602109  853.84045554  854.07575611
  523.38703697 1061.99302949  943.28361528  933.64826142  657.88713754
 1355.0055712  1125.52363599  945.07387491  893.30917863  677.06450629
 1295.98067646 1022.58201128  917.40748316  850.26457359  664.19477951
 1209.46931051  973.62970678  853.94132498  766.98240265  547.47121603
 1178.57364423  955.66594935  872.95218196  781.30935349  564.56653455
 1205.92632552 1017.75203768  880.14559195  889.38800429  581.64928947]


In [144]:
# Lasso
model = Lasso(alpha=0.01)
model.fit(ln_X, ln_y)

lss_preds = model.predict(ln_X_test)
print(lss_preds)

[ 957.6385332   875.84829604  625.52243998 1254.57704469  983.62970928
  947.17135341  908.10933207  664.05922081 1238.13548892  980.61430338
  774.15152008 1296.211647   1090.94285065  974.38573562  872.19745757
  639.92562641 1222.16322481 1014.91929326  856.36491273  853.91871154
  522.08844233 1060.15138756  943.52511448  934.44553051  657.59897875
 1354.5949276  1125.88196913  944.69166973  893.50737352  675.31694489
 1295.80815924 1022.25645211  917.60908298  849.80891542  663.47950817
 1209.73084922  973.27880888  853.46447057  766.9353761   547.28595435
 1178.411751    955.87410842  872.93432824  781.44099955  564.87508618
 1205.45837616 1017.89915486  879.60132143  888.75675426  581.01019375]


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5248673.893245975, tolerance: 5277.687601163757
  model = cd_fast.enet_coordinate_descent(


In [145]:
model = ElasticNet(alpha=0.01)
model.fit(ln_X, ln_y)

eln_preds = model.predict(ln_X_test)
print(eln_preds)

[ 956.57862416  877.09241525  624.55416079 1251.01142762  984.0112047
  945.48425004  910.53853208  664.40655585 1233.785704    981.27394447
  777.22166909 1294.15514173 1092.27366674  973.07870047  871.72620208
  639.21268345 1217.79744794 1016.85936504  853.38855855  854.71546539
  526.35081081 1064.84961162  944.67076627  938.09297681  660.67066279
 1347.92301913 1131.85363544  946.93031134  898.01359127  678.20935942
 1296.51679126 1025.93857698  918.12647992  852.5249026   658.75086739
 1208.10970369  975.5317372   851.82941215  768.08310264  548.18169677
 1177.05565661  959.42743115  873.02655045  782.22617584  565.04596449
 1203.43031332 1022.42234599  878.71940776  884.78664264  582.12888527]


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5505433.788163523, tolerance: 5277.687601163757
  model = cd_fast.enet_coordinate_descent(


In [146]:
# XGBR
xgbr = XGBRegressor(n_estimators=300, colsample_bylevel=0.5, learning_rate=0.08, 
                    colsample_bytree=0.5, max_depth=2, objective="reg:squarederror")
xgbr.fit(ln_X, ln_y)

xgbr_preds = xgbr.predict(ln_X_test)
print(xgbr_preds)

[1017.7718   940.05743  675.86237 1268.9152  1063.0638   959.72235
  972.23206  699.18896 1284.1606  1061.7432   731.4684  1280.241
 1104.6763  1030.8915   925.1685   670.726   1263.2375   977.57635
  858.23456  817.51556  585.6231  1072.1274  1001.0339   925.42926
  654.9794  1327.0253  1120.7433   992.6855   912.17456  716.3607
 1273.2975  1048.3485  1016.96313  891.53235  654.717   1262.0873
 1022.2643   887.6341   816.13544  587.7983  1252.2306  1031.0671
  953.0747   854.09546  599.22375 1235.6232  1051.6527   969.90186
  877.9064   589.31854]


In [147]:
# LGBR
lgbr = LGBMRegressor(n_estimator=400)
lgbr.fit(ln_X, ln_y, early_stopping_rounds=150, 
         eval_metric="logloss", eval_set=[(ln_X, ln_y)], verbose=True)

lgbr_preds = lgbr.predict(ln_X_test)
print(lgbr_preds)

[LightGBM] [Warning] Unknown parameter: n_estimator
[1]	training's l2: 37429.9
[2]	training's l2: 32116
[3]	training's l2: 27749.8
[4]	training's l2: 24163.6
[5]	training's l2: 21217.5
[6]	training's l2: 18770.1
[7]	training's l2: 16745.2
[8]	training's l2: 14990.6
[9]	training's l2: 13558.9
[10]	training's l2: 12355.4
[11]	training's l2: 11340.8
[12]	training's l2: 10525.9
[13]	training's l2: 9780.48
[14]	training's l2: 9169.56
[15]	training's l2: 8431.48
[16]	training's l2: 7807.15
[17]	training's l2: 7303.27
[18]	training's l2: 6862.75
[19]	training's l2: 6501.16
[20]	training's l2: 6173.32
[21]	training's l2: 5886.06
[22]	training's l2: 5629.16
[23]	training's l2: 5393.61
[24]	training's l2: 5183.2
[25]	training's l2: 4987.45
[26]	training's l2: 4825.13
[27]	training's l2: 4662.93
[28]	training's l2: 4537.89
[29]	training's l2: 4400.94
[30]	training's l2: 4288.97
[31]	training's l2: 4158.39


C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[32]	training's l2: 4035.87
[33]	training's l2: 3923.19
[34]	training's l2: 3801.21
[35]	training's l2: 3700.65
[36]	training's l2: 3616.4
[37]	training's l2: 3523.08
[38]	training's l2: 3433.6
[39]	training's l2: 3336.15
[40]	training's l2: 3257.99
[41]	training's l2: 3167.84
[42]	training's l2: 3098.93
[43]	training's l2: 3030.16
[44]	training's l2: 2947.01
[45]	training's l2: 2886.13
[46]	training's l2: 2827.57
[47]	training's l2: 2758.69
[48]	training's l2: 2705.28
[49]	training's l2: 2641.93
[50]	training's l2: 2581.69
[51]	training's l2: 2524.75
[52]	training's l2: 2472.61
[53]	training's l2: 2426.01
[54]	training's l2: 2370.08
[55]	training's l2: 2324.51
[56]	training's l2: 2281.04
[57]	training's l2: 2242.1
[58]	training's l2: 2203.13
[59]	training's l2: 2162.76
[60]	training's l2: 2119.86
[61]	training's l2: 2081.73
[62]	training's l2: 2044.58
[63]	training's l2: 2009.58
[64]	training's l2: 1973.72
[65]	training's l2: 1938.89
[66]	training's l2: 1906.19
[67]	training's l2: 187

In [148]:
submission = pd.DataFrame({"datetime":ln_test["datetime"], "lr":lr_preds, "svr":svr_preds, "rdg":rdg_preds, "lss":lss_preds,
                          "eln":eln_preds, "xgbr":xgbr_preds, "lgbr":lgbr_preds})
submission

,datetime,lr,svr,rdg,lss,eln,xgbr,lgbr
0,2021-01-27,971.210293,874.257404,958.787725,957.638533,956.578624,1017.771790,1032.397669
1,2021-01-28,888.139488,818.273389,878.628931,875.848296,877.092415,940.057434,942.417894
2,2021-01-29,637.776877,586.386870,627.312017,625.522440,624.554161,675.862366,603.318238
3,2021-02-01,1264.325627,1192.234115,1256.871512,1254.577045,1251.011428,1268.915161,1234.915487
4,2021-02-02,996.002935,1005.257924,984.930887,983.629709,984.011205,1063.063843,1008.025788
5,2021-02-03,958.966284,899.858172,946.997422,947.171353,945.484250,959.722351,982.341067
6,2021-02-04,918.453018,895.128671,908.695035,908.109332,910.538532,972.232056,932.269292
7,2021-02-05,671.881073,631.229221,663.245298,664.059221,664.406556,699.188965,636.328984
8,2021-02-08,1251.668779,1239.360621,1238.751997,1238.135489,1233.785704,1284.160645,1316.953810
9,2021-02-09,990.265371,979.334800,979.033590,980.614303,981.273944,1061.743164,1091.382292


In [149]:
submission.to_csv("lunch_number_submission.csv", encoding='utf-8', index=False)

---
---
---
---
---

# 구내식당 석식 인원 예측 모델

In [107]:
dn_train = pd.read_csv("../byun/data/dinner_df_encoding.csv", encoding="utf-8")
dn_train

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_dinner_N,new_dinner_Y
0,2016-02-01,2016,2,1,2601,2401,50,150,238,0,...,1,0,1,0,0,0,1,0,1,0
1,2016-02-02,2016,2,2,2601,2378,50,173,319,0,...,1,0,0,0,1,0,1,0,1,0
2,2016-02-03,2016,2,3,2601,2365,56,180,111,0,...,1,0,0,0,0,1,1,0,1,0
3,2016-02-04,2016,2,4,2601,2277,104,220,355,0,...,1,0,0,1,0,0,1,0,1,0
4,2016-02-05,2016,2,5,2601,2142,278,181,34,0,...,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2021,1,20,2983,2319,75,198,4,391,...,1,0,0,0,0,1,1,0,1,0
1201,2021-01-21,2021,1,21,2983,2309,92,231,462,351,...,1,0,0,1,0,0,1,0,1,0
1202,2021-01-22,2021,1,22,2983,2177,255,248,1,303,...,1,1,0,0,0,0,1,0,1,0
1203,2021-01-25,2021,1,25,2983,2396,107,153,616,327,...,1,0,1,0,0,0,1,0,1,0


In [108]:
dn_train.isnull().sum()
dn_train = dn_train.dropna()

## 선형회귀(일반) - train: 52.53%, test: 48.01%

In [112]:
dn_X = dn_train.drop(columns=["datetime", "dinner_number"], axis=1)

dn_y = dn_train["dinner_number"]

dn_y = dn_y.to_numpy()


lr = LinearRegression().fit(dn_X, dn_y)
print("훈련 점수 : ", lr.score(dn_X, dn_y))
print("========================================================")
dn_X_train, dn_X_test, dn_y_train, dn_y_test = train_test_split(dn_X, dn_y, test_size=0.2, random_state=0)
lr = LinearRegression().fit(dn_X_train, dn_y_train)

print("훈련 점수 : ", lr.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", lr.score(dn_X_test, dn_y_test))

훈련 점수 :  0.5249828965311587
훈련 점수 :  0.5252706818889732
테스트 점수 :  0.4801202488360182


## 릿지회귀모델 - train: 53.11%, test: 50.38%

In [94]:
ridge = Ridge().fit(dn_X_train, dn_y_train) 

print("훈련 점수 : ", ridge.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", ridge.score(dn_X_test, dn_y_test))
print("========================================================")

ridge10 = Ridge(alpha=100).fit(dn_X_train, dn_y_train) 

print("훈련 점수 : ", ridge10.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", ridge10.score(dn_X_test, dn_y_test))

print("========================================================")
ridge01 = Ridge(alpha=0.000000001).fit(dn_X_train, dn_y_train) 

print("훈련 점수 : ", ridge01.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", ridge01.score(dn_X_test, dn_y_test))

훈련 점수 :  0.5311031341997445
테스트 점수 :  0.5038654429839337
훈련 점수 :  0.5215506664632992
테스트 점수 :  0.5000294643047009
훈련 점수 :  0.5311097299673089
테스트 점수 :  0.5034655621849163


## 라쏘회귀모델 - train: 53.10%, test: 50.47%

In [95]:

lasso = Lasso().fit(dn_X_train, dn_y_train) #Lasso(alpha=1) 디폴트

print("훈련 점수 : ", lasso.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", lasso.score(dn_X_test, dn_y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso.coef_ != 0)) 
print("========================================================")
lasso001 = Lasso(alpha=0.01).fit(X_train, y_train)

print("훈련 점수 : ", lasso001.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", lasso001.score(dn_X_test, dn_y_test))

print("사용한 특성의 수 : ", np.sum(lasso001.coef_ != 0))
print("========================================================")
lasso00001 = Lasso(alpha=0.000000001).fit(dn_X_train, dn_y_train)

print("훈련 점수 : ", lasso00001.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", lasso00001.score(dn_X_test, dn_y_test)) 

print("사용한 특성의 수 : ", np.sum(lasso00001.coef_ != 0))

# Lasso: lower alpha makes greater score

훈련 점수 :  0.52748563774949
테스트 점수 :  0.49556220016851027
사용한 특성의 수 :  16
훈련 점수 :  0.5309736286294525
테스트 점수 :  0.5047508824135023
사용한 특성의 수 :  25
훈련 점수 :  0.530983084214575
테스트 점수 :  0.5046886765270251
사용한 특성의 수 :  27


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 37772.5374730397, tolerance: 1899.8009273388775
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4273734.057619, tolerance: 1899.8009273388775
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4455193.840111321, tolerance: 1899.8009273388775
  model = cd_fast.enet_coordinate_descent(


## 엘라스틱넷 - train: 53.08%, test: 50.57%

In [97]:
model = ElasticNet(alpha=0.01)
model.fit(dn_X_train, dn_y_train)

print("훈련 점수 : ", model.score(dn_X_train, dn_y_train))
print("테스트 점수 : ", model.score(dn_X_test, dn_y_test))

훈련 점수 :  0.5308462284648889
테스트 점수 :  0.5057277857833855


C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4368649.036843837, tolerance: 1899.8009273388775
  model = cd_fast.enet_coordinate_descent(


## SVR model - train: 47.37%, test: 48.92%

- Normal
- Kernel("linear" is best)
- StandardScaler // MinMaxScaler

In [98]:
model = SVR()  # default is rbf
model.fit(dn_X_train, dn_y_train)

print("rbf 학습 결과 : ", model.score(dn_X_train, dn_y_train))
print("rbf 테스트 결과 : ", model.score(dn_X_test, dn_y_test))
print('===================================================')
# Too bad

linear_svr = SVR(kernel="linear")
linear_svr.fit(dn_X_train, dn_y_train)

print("linear 학습 결과 : ", linear_svr.score(dn_X_train, dn_y_train))
print("linear 테스트 결과 : ", linear_svr.score(dn_X_test, dn_y_test))

print('===================================================')

linear_svr = SVR(kernel="linear", C=10, gamma='scale')
linear_svr.fit(dn_X_train, dn_y_train)

print("C=10 학습 결과 : ", linear_svr.score(dn_X_train, dn_y_train))
print("C=10 테스트 결과 : ", linear_svr.score(dn_X_test, dn_y_test))

print('===================================================')

### 표준화 해보자

scaler = StandardScaler()
dn_X_train_std = scaler.fit_transform(dn_X_train)
dn_X_test_std = scaler.transform(dn_X_test)

model = SVR(kernel='linear')
model.fit(dn_X_train_std, dn_y_train)

print("std 학습 결과 : ", model.score(dn_X_train_std, dn_y_train))
print("std 테스트 결과 : ", model.score(dn_X_test_std, dn_y_test))
print('===================================================')
## 표준화만 했는데도 성능이 좋아짐

model = SVR(kernel='linear', C=10)
model.fit(dn_X_train_std, dn_y_train)

print("std 학습 결과 : ", model.score(dn_X_train_std, dn_y_train))
print("std 테스트 결과 : ", model.score(dn_X_test_std, dn_y_test))
print('===================================================')
model = SVR(kernel='linear', C=10, gamma='auto')
model.fit(dn_X_train_std, dn_y_train)

print("std 학습 결과 : ", model.score(dn_X_train_std, dn_y_train))
print("std 테스트 결과 : ", model.score(dn_X_test_std, dn_y_test))
print('===================================================')


rbf 학습 결과 :  0.010646474562626795
rbf 테스트 결과 :  0.016858563236203272
linear 학습 결과 :  0.46862264208574655
linear 테스트 결과 :  0.4801582231467556
C=10 학습 결과 :  0.4107284966793153
C=10 테스트 결과 :  0.4156136971305534
std 학습 결과 :  0.4496245524967396
std 테스트 결과 :  0.4889316348352648
std 학습 결과 :  0.4676095195114729
std 테스트 결과 :  0.49049424946629216
std 학습 결과 :  0.4676095195114729
std 테스트 결과 :  0.49049424946629216


In [134]:
# best 값 적용 (linear kernel, C=10, gamma=scale, epsilon=10)
model = SVR(kernel="linear", C=10.0, gamma="scale", epsilon=10.0)
model.fit(dn_X_train_std, dn_y_train)

print("학습 결과 : ", model.score(dn_X_train_std, dn_y_train))
print("테스트 결과 : ", model.score(dn_X_test_std, dn_y_test))

학습 결과 :  0.47365202201080825
테스트 결과 :  0.4892416502784851


## Gradient Boosting Model

- XGBoostRegressor(overfitting)
- LightGBMRegressor(overfitting)

### XGBoostRegressor - train: 76.51%, test: 63.42%, testscore_mean: 43.65%

In [99]:
dn_X_train, dn_X_test, dn_y_train, dn_y_test = train_test_split(dn_X, dn_y, test_size=.2, random_state=42)

xgbr = XGBRegressor(n_estimators=300, colsample_bylevel=0.5, learning_rate=0.08, 
                    colsample_bytree=0.5, max_depth=2, objective="reg:squarederror")
xgbr.fit(dn_X_train, dn_y_train)

print("훈련 정확도 : ", xgbr.score(dn_X_train, dn_y_train))
print("테스트 정확도 : ", xgbr.score(dn_X_test, dn_y_test))

cross_val = cross_validate(estimator=xgbr, X=dn_X, y=dn_y, cv=10)

print("테스트 정확도 평균 : ", cross_val["test_score"].mean())

훈련 정확도 :  0.7650666744957487
테스트 정확도 :  0.6342214140882214
테스트 정확도 평균 :  0.4365752466303988


### LightGBM Regressor - train: 84.80%, test: 65.06%, testscore_mean: 46.38%

In [100]:
dn_X_train, dn_X_test, dn_y_train, dn_y_test = train_test_split(dn_X, dn_y, test_size=.2, random_state=42)

lgbr = LGBMRegressor(n_estimator=400)
lgbr.fit(dn_X_train, dn_y_train, early_stopping_rounds=150, 
         eval_metric="logloss", eval_set=[(dn_X_test, dn_y_test)], verbose=True)

print("훈련 정확도 : ", lgbr.score(dn_X_train, dn_y_train))
print("테스트 정확도 : ", lgbr.score(dn_X_test, dn_y_test))

cross_val = cross_validate(estimator=lgbr, X=dn_X, y=dn_y, cv=10)
print("테스트 정확도 평균 : ", cross_val["test_score"].mean())

C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: n_estimator
[1]	valid_0's l2: 14316.2
[2]	valid_0's l2: 12730.9
[3]	valid_0's l2: 11501.9
[4]	valid_0's l2: 10409.7
[5]	valid_0's l2: 9529.74
[6]	valid_0's l2: 8724.53
[7]	valid_0's l2: 8119.98
[8]	valid_0's l2: 7633.71
[9]	valid_0's l2: 7239.23
[10]	valid_0's l2: 6909.16
[11]	valid_0's l2: 6697.32
[12]	valid_0's l2: 6475.28
[13]	valid_0's l2: 6297.83
[14]	valid_0's l2: 6149.87
[15]	valid_0's l2: 6109.82
[16]	valid_0's l2: 6065.56
[17]	valid_0's l2: 6021.12
[18]	valid_0's l2: 5957.6
[19]	valid_0's l2: 5906.17
[20]	valid_0's l2: 5871.78
[21]	valid_0's l2: 5834.09
[22]	valid_0's l2: 5815.3
[23]	valid_0's l2: 5809.37
[24]	valid_0's l2: 5808.05
[25]	valid_0's l2: 5775.76
[26]	valid_0's l2: 5716.66
[27]	valid_0's l2: 5712.56
[28]	valid_0's l2: 5686.09
[29]	valid_0's l2: 5698.89
[30]	valid_0's l2: 5711.65
[31]	valid_0's l2: 5731.09
[32]	valid_0's l2: 5744.64
[33]	valid_0's l2: 5722.47
[34]	valid_0's l2: 5722.52
[35]	valid_0's l2: 5755.98
[36]	valid_0's

# Dinner Test Data

In [105]:
dn_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 0 to 1204
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   1203 non-null   int64  
 1   month                  1203 non-null   int64  
 2   date                   1203 non-null   int64  
 3   worker_number          1203 non-null   int64  
 4   real_number            1203 non-null   int64  
 5   vacation_number        1203 non-null   int64  
 6   biztrip_number         1203 non-null   int64  
 7   overtime_number        1203 non-null   int64  
 8   telecom_number         1203 non-null   int64  
 9   temperature            1203 non-null   float64
 10  rain                   1203 non-null   float64
 11  wind                   1203 non-null   float64
 12  humidity               1203 non-null   float64
 13  discomfort_index       1203 non-null   float64
 14  perceived_temperature  1203 non-null   float64
 15  dinn

In [101]:
dn_test = pd.read_csv("../byun/data/dinner_test_encoding.csv", encoding="utf-8")
dn_test

,datetime,year,month,date,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,...,season_winter,weekdays_friday,weekdays_monday,weekdays_thursday,weekdays_tuesday,weekdays_wednesday,vacation_N,vacation_Y,new_dinner_N,new_dinner_Y
0,2021-01-27,2021,1,27,2983,2355.0,88,182,5,358.0,...,1,0,0,0,0,1,1,0,0,1
1,2021-01-28,2021,1,28,2983,2319.0,104,212,409,348.0,...,1,0,0,1,0,0,1,0,1,0
2,2021-01-29,2021,1,29,2983,2170.0,270,249,0,294.0,...,1,1,0,0,0,0,1,0,1,0
3,2021-02-01,2021,2,1,2924,2340.0,108,154,538,322.0,...,1,0,1,0,0,0,1,0,1,0
4,2021-02-02,2021,2,2,2924,2362.0,62,186,455,314.0,...,1,0,0,0,1,0,1,0,1,0
5,2021-02-03,2021,2,3,2924,2380.0,59,199,5,286.0,...,1,0,0,0,0,1,1,0,1,0
6,2021-02-04,2021,2,4,2924,2364.0,61,211,476,288.0,...,1,0,0,1,0,0,1,0,1,0
7,2021-02-05,2021,2,5,2924,2247.0,169,252,0,256.0,...,1,1,0,0,0,0,1,0,1,0
8,2021-02-08,2021,2,8,2924,2333.0,88,174,690,329.0,...,1,0,1,0,0,0,1,0,1,0
9,2021-02-09,2021,2,9,2924,2318.0,94,183,542,329.0,...,1,0,0,0,1,0,1,0,1,0


In [113]:
summer = []
fall = []

for i in range(len(ln_test)):
    summer.append(0)
    fall.append(0)
    
dn_test['season_summer'] = summer
dn_test['season_fall'] = fall

columns = ['datetime', 'year', 'month', 'date', 'worker_number', 'real_number',
       'vacation_number', 'biztrip_number', 'overtime_number',
       'telecom_number', 'temperature', 'rain', 'wind', 'humidity',
       'discomfort_index', 'perceived_temperature', 'dinner_rice',
       'season_fall','season_spring', 'season_summer', 'season_winter', 'weekdays_friday', 'weekdays_monday',
       'weekdays_thursday', 'weekdays_tuesday', 'weekdays_wednesday',
       'vacation_N', 'vacation_Y', 'new_dinner_N', 'new_dinner_Y']

dn_test.coulmns = columns


dn_X_test= dn_test[['year', 'month', 'date', 'worker_number', 'real_number',
       'vacation_number', 'biztrip_number', 'overtime_number',
       'telecom_number', 'temperature', 'rain', 'wind', 'humidity',
       'discomfort_index', 'perceived_temperature', 'dinner_rice',
       'season_fall','season_spring', 'season_summer', 'season_winter', 'weekdays_friday', 'weekdays_monday',
       'weekdays_thursday', 'weekdays_tuesday', 'weekdays_wednesday',
       'vacation_N', 'vacation_Y', 'new_dinner_N', 'new_dinner_Y']]
dn_X_test.shape
dn_X_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   50 non-null     int64  
 1   month                  50 non-null     int64  
 2   date                   50 non-null     int64  
 3   worker_number          50 non-null     int64  
 4   real_number            50 non-null     float64
 5   vacation_number        50 non-null     int64  
 6   biztrip_number         50 non-null     int64  
 7   overtime_number        50 non-null     int64  
 8   telecom_number         50 non-null     float64
 9   temperature            50 non-null     float64
 10  rain                   50 non-null     float64
 11  wind                   50 non-null     float64
 12  humidity               50 non-null     float64
 13  discomfort_index       50 non-null     float64
 14  perceived_temperature  50 non-null     float64
 15  dinner_r

In [150]:
# LR
model = LinearRegression()
model.fit(dn_X, dn_y)

lr_preds = model.predict(dn_X_test)
print(lr_preds)

# SVR
model = SVR(kernel='linear', C=10, gamma='scale', epsilon=10)
model.fit(dn_X, dn_y)

svr_preds = model.predict(dn_X_test)
print(svr_preds)

# Ridge
model = Ridge(alpha=0.01)
model.fit(dn_X, dn_y)

rdg_preds = model.predict(dn_X_test)
print(rdg_preds)

# Lasso
model = Lasso(alpha=0.01)
model.fit(dn_X, dn_y)

lss_preds = model.predict(dn_X_test)
print(lss_preds)

model = ElasticNet(alpha=0.01)
model.fit(dn_X, dn_y)

eln_preds = model.predict(dn_X_test)
print(eln_preds)

# XGBR
xgbr = XGBRegressor(n_estimators=300, colsample_bylevel=0.5, learning_rate=0.08, 
                    colsample_bytree=0.5, max_depth=2, objective="reg:squarederror")
xgbr.fit(dn_X, dn_y)

xgbr_preds = xgbr.predict(dn_X_test)
print(xgbr_preds)

# LGBR
lgbr = LGBMRegressor(n_estimator=400)
lgbr.fit(dn_X, dn_y, early_stopping_rounds=150, 
         eval_metric="logloss", eval_set=[(dn_X, dn_y)], verbose=True)

lgbr_preds = lgbr.predict(dn_X_test)
print(lgbr_preds)

submission = pd.DataFrame({"datetime":dn_test["datetime"], "lr":lr_preds, "svr":svr_preds, "rdg":rdg_preds, "lss":lss_preds,
                          "eln":eln_preds, "xgbr":xgbr_preds, "lgbr":lgbr_preds})
submission

[273.37752533 403.3929882  287.31393433 524.86919785 491.33611679
 311.69338989 474.35664177 335.70052338 572.89348221 489.20330238
 192.68354225 581.93858147 579.60761452 298.78477478 459.6498661
 299.28641891 570.52069855 564.23350525 256.50042534 431.54978943
 212.78924561 617.36184883 332.25163841 563.79376984 364.53903008
 673.21404839 693.32070351 340.94282341 523.56917763 348.41226196
 636.84770584 556.60052681 313.84314728 454.32652283 329.60712242
 576.82665825 540.16337204 278.15584946 410.94876099 301.50921631
 536.51916313 513.54441643 277.62875175 454.30543518 347.30427933
 590.26313591 567.87563896 322.06863213 473.87764359 333.50586891]
[340.92488119 443.0294345  368.23303056 479.49510771 518.37871444
 325.65971482 485.45741517 329.30848438 591.10544801 495.5836604
 194.81468892 554.76236102 583.29126465 344.67153695 508.98379815
 312.24524498 567.54339454 624.78399065 312.31721619 443.95581678
 233.27634239 597.85221732 338.6657942  530.3054525  334.80966681
 640.306641

C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5060362.750242895, tolerance: 2332.14158719867
  model = cd_fast.enet_coordinate_descent(
C:\Users\tagos71\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5258590.829298117, tolerance: 2332.14158719867
  model = cd_fast.enet_coordinate_descent(


[277.33120668 403.63504529 288.80739504 521.29844141 491.58525556
 312.70696388 474.25718309 335.50665887 570.05029501 489.93991436
 195.94560928 579.65505798 573.22367325 302.4719925  458.84803786
 301.08888854 569.62711091 561.90840838 262.83295893 431.59187389
 217.27930501 612.71471517 337.38861482 561.67125895 365.60984584
 666.68328822 686.35121987 342.44829914 519.23853992 347.71841017
 633.65326095 557.64138268 317.87986938 460.19463829 328.58873143
 572.15882553 538.39811797 281.43255181 414.45642936 299.41926943
 534.92731522 513.83155801 281.35657813 454.08080658 348.59125748
 586.65576813 566.02775225 323.58313682 477.61848155 330.57785766]
[219.92459 426.96176 274.51254 548.71924 500.42877 389.3174  522.0713
 367.67084 630.8735  535.3345  201.90178 667.43964 646.58    409.45853
 536.20776 394.30542 641.7509  630.6214  335.1358  477.49057 241.81174
 614.7996  407.58734 580.5477  371.88773 766.1489  750.1447  409.4833
 560.9836  353.7895  691.16785 608.86414 390.73123 508.86

C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tagos71\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[47]	training's l2: 2085.75
[48]	training's l2: 2049.57
[49]	training's l2: 2010.25
[50]	training's l2: 1966.93
[51]	training's l2: 1926.53
[52]	training's l2: 1893.66
[53]	training's l2: 1857.08
[54]	training's l2: 1821.94
[55]	training's l2: 1790.7
[56]	training's l2: 1756.34
[57]	training's l2: 1727.41
[58]	training's l2: 1697.59
[59]	training's l2: 1668.06
[60]	training's l2: 1637.42
[61]	training's l2: 1607.48
[62]	training's l2: 1574.29
[63]	training's l2: 1541.39
[64]	training's l2: 1516.13
[65]	training's l2: 1488.96
[66]	training's l2: 1460.87
[67]	training's l2: 1437.53
[68]	training's l2: 1413.63
[69]	training's l2: 1387.6
[70]	training's l2: 1362.22
[71]	training's l2: 1343.47
[72]	training's l2: 1317.61
[73]	training's l2: 1296.84
[74]	training's l2: 1272.42
[75]	training's l2: 1245.22
[76]	training's l2: 1223.04
[77]	training's l2: 1201.11
[78]	training's l2: 1180.95
[79]	training's l2: 1154.9
[80]	training's l2: 1135.59
[81]	training's l2: 1118.48
[82]	training's l2: 11

,datetime,lr,svr,rdg,lss,eln,xgbr,lgbr
0,2021-01-27,273.377525,340.924881,275.438056,275.633119,277.331207,219.924591,113.740141
1,2021-01-28,403.392988,443.029435,404.988125,402.828646,403.635045,426.961761,398.153688
2,2021-01-29,287.313934,368.233031,290.810185,289.365524,288.807395,274.512543,326.275962
3,2021-02-01,524.869198,479.495108,521.517556,519.710768,521.298441,548.719238,493.110988
4,2021-02-02,491.336117,518.378714,491.100578,490.005855,491.585256,500.428772,377.357979
5,2021-02-03,311.693390,325.659715,311.152983,311.256823,312.706964,389.317413,430.931644
6,2021-02-04,474.356642,485.457415,473.674773,473.067046,474.257183,522.071289,433.705055
7,2021-02-05,335.700523,329.308484,334.821591,335.233778,335.506659,367.670837,395.583701
8,2021-02-08,572.893482,591.105448,569.522263,568.921950,570.050295,630.873474,616.946593
9,2021-02-09,489.203302,495.583660,487.861356,488.881215,489.939914,535.334473,470.382129


In [151]:
submission.to_csv("dinner_number_submission.csv", encoding='utf-8', index=False)